In [12]:
%load_ext autoreload
%autoreload 2

import numpy as np
from tqdm import tqdm

import sqlalchemy as sqla
from sqlalchemy import create_engine, Column, and_
from sqlalchemy.orm import Session

from ultrack.core.database import NodeDB
from ultrack.core.export.utils import solution_dataframe_from_sql
from tifffile import imread

import sys
sys.path.append('..')
from tracks_interactions.db.db_model import Base, CellDB, TrackDB
from tracks_interactions.db.db_functions import add_track_ids_to_tracks_df, calculate_cell_signals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# create a new database

new_db_path = r'D:\kasia\tracking\E6_exp\sample_data\sample_db_2tables.db'

engine = create_engine(f'sqlite:///{new_db_path}')

# creates a table
Base.metadata.create_all(engine) 

In [3]:
# get engine for the original database

org_db_path = r'D:\kasia\tracking\E6_exp\sample_data\data.db'
engine_org = sqla.create_engine(f'sqlite:///{org_db_path}')

In [4]:
# get a solution in a form of a dataframe

df = solution_dataframe_from_sql(f'sqlite:///{org_db_path}')
df = add_track_ids_to_tracks_df(df)

df.reset_index(inplace=True)
df

c:\Users\kmk280\.conda\envs\tracking\lib\site-packages\numba\typed\typeddict.py:39: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  return d[key]


,id,parent_id,t,z,y,x,track_id,parent_track_id,root
0,3000001,2000002,2,0.0,30.0,148.0,1,-1,1.0
1,6000005,5000003,5,0.0,30.0,178.0,1,-1,1.0
2,6000006,5000006,5,0.0,42.0,417.0,3,-1,3.0
3,6000007,5000007,5,0.0,15.0,469.0,2,-1,2.0
4,6000008,5000008,5,0.0,17.0,533.0,5,-1,5.0
...,...,...,...,...,...,...,...,...,...
1288,18000082,17000081,17,0.0,962.0,977.0,72,71,71.0
1289,18000083,17000076,17,0.0,968.0,13.0,88,86,86.0
1290,18000084,17000080,17,0.0,970.0,501.0,80,-1,80.0
1291,18000085,17000079,17,0.0,973.0,787.0,66,-1,66.0


## Create a cells table

In [5]:
# that has to be changed to operate on the original database
# because at the moment objects not assigned to a track are not saved in the database
# the consideration is what if multiple segmentations were given to ultrack and
# there are multiple possible objects for a single cell ???

def add_cell(row):

        global session
        global session_db_org
        
        cell = CellDB(id = row['id'],
                    t =row['t'],
                    track_id = row['track_id'],
                    row = row['y'],
                    col = row['x'])
        
        # get a mask of this cell
        cell_obj = session_db_org.query(NodeDB).filter(NodeDB.id==row['id']).first()

        cell.mask = cell_obj.pickle.mask
        
        cell.bbox_0 = int(cell_obj.pickle.bbox[0])
        cell.bbox_1 = int(cell_obj.pickle.bbox[1])
        cell.bbox_2 = int(cell_obj.pickle.bbox[2])
        cell.bbox_3 = int(cell_obj.pickle.bbox[3])

        session.add(cell)

In [6]:
# create a table of cells

tqdm.pandas(desc="Progress")

session_db_org = Session(engine_org)
session = Session(engine)

df.progress_apply(add_cell, axis=1)

session.commit()

session_db_org.close()
session.close()

Progress: 100%|██████████| 1293/1293 [00:00<00:00, 2527.90it/s]


## Add signals

In [10]:
ch0_path = r'D:\kasia\tracking\E6_exp\sample_data\ch1.tif'
ch1_path = r'D:\kasia\tracking\E6_exp\sample_data\ch2.tif'

ch0_org = imread(ch0_path)
ch1_org = imread(ch1_path)

In [11]:
ch0_org.shape

(20, 1000, 1000)

In [15]:
cell

1000004 from frame 0 with track_id 1 at (29,156)

In [17]:
# for exp6 around 25 min

session = Session(engine)

for frame in tqdm(range(ch0_org.shape[0])):

    cells = session.query(CellDB).filter(CellDB.t==frame).all()
    ch0 = ch0_org[frame]
    ch1 = ch1_org[frame]

    for cell in cells:

        # Calculate cell measurements for each cell
        new_signals = calculate_cell_signals(cell, [ch0, ch1])
        
        # Update the signals field with the new JSON data
        cell.signals = new_signals
        
    # Commit changes to the database
    session.commit()

session.close()

100%|██████████| 20/20 [00:00<00:00, 21.46it/s]


## Create a tracks table

In [18]:
df_tracks = df.groupby(['track_id','parent_track_id' ,'root']).agg({'t':['min','max']})
df_tracks.reset_index(inplace=True)
df_tracks.columns = ['_'.join(col).strip('_') for col in df_tracks.columns.values]
df_tracks

,track_id,parent_track_id,root,t_min,t_max
0,1,-1,1.0,0,11
1,2,-1,2.0,0,7
2,3,-1,3.0,0,19
3,4,-1,4.0,0,7
4,5,-1,5.0,0,9
...,...,...,...,...,...
84,85,-1,85.0,12,19
85,86,-1,86.0,4,14
86,87,86,86.0,15,19
87,88,86,86.0,15,19


In [19]:
def add_track(row):

        global session
        
        track = TrackDB(track_id = row['track_id'],
                        parent_track_id = row['parent_track_id'],
                        root = row['root'],
                        t_begin = row['t_min'],
                        t_end = row['t_max'])
        

        session.add(track)

In [20]:
# create a table of tracks

session = Session(engine)  

df_tracks.apply(add_track, axis=1)

session.commit()

session.close()

### Get a single track by track_id

In [12]:
with Session(engine) as session:
    t = session.query(TrackDB).get(1)

t

C:\Users\kmk280\AppData\Local\Temp\ipykernel_41848\4267526768.py:2: LegacyAPIWarning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  t = session.query(TrackDB).get(1)


Track 1 from 0 to 11